In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
print(tf.__version__)

2.3.0


In [3]:
import nltk
from nltk.tokenize import word_tokenize

In [4]:
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

In [5]:
lemmatizer=WordNetLemmatizer()
hm_lines=10000000

In [6]:
def create_lexicon(pos,neg):
    lexicon=[]
    for fi in [pos,neg]:
        with open(fi,'r') as f:
            contents=f.readlines()
            for l in contents[:hm_lines]:
                all_words=word_tokenize(l)
                lexicon+=list(all_words)
    lexicon=[lemmatizer.lemmatize(i) for i in lexicon]
    w_counts=Counter(lexicon)
    #w_counts={'the':523243,"and":5435}
    l2=[]
    for w in w_counts:
        if(1000>w_counts[w]>50):
            l2.append(w)
    print(len(l2))
    return l2   

In [7]:
def sample_handling(sample,lexicon,classification):
    featureset=[]
    with open(sample,'r') as f:
        contents=f.readlines()
        for l in contents[:hm_lines]:
            current_words=word_tokenize(l.lower())
            current_words=[lemmatizer.lemmatize(i) for i in current_words]
            features=np.zeros(len(lexicon))
            for word in current_words:
                if(word.lower() in lexicon):
                    index_value=lexicon.index(word.lower())
                    features[index_value]+=1
            features=list(features)
            featureset.append([features,classification])
    return featureset

In [8]:
def create_feature_sets_labels(pos,neg,test_size=0.1):
    lexicon=create_lexicon(pos,neg)
    features=[]
    features+=sample_handling('pos.txt',lexicon,[1,0])
    features+=sample_handling('neg.txt',lexicon,[0,1])
    random.shuffle(features)
    features=np.array(features)
    testing_size=int(test_size*len(features))
    #[[0 1 1 0 1]],[0,1]]] ->features look like this 
    #features[:,0] means it takes x_train as [0 1 1 0 1]
    x_train=list(features[:,0][:-testing_size])
    y_train=list(features[:,1][:-testing_size])
    x_test=list(features[:,0][-testing_size])
    y_test=list(features[:,1][-testing_size])
    return x_train,x_test,y_train,y_test
    

In [9]:
if(__name__=='__main__'):
    x_train,x_test,y_train,y_test=create_feature_sets_labels('pos.txt','neg.txt')
    with open('sentiment_set.pickle','wb') as f:
        pickle.dump([x_train,x_test,y_train,y_test],f)

423


In [10]:
#now we have dataset in vector form 
#now we have to introduce this data to deep neural network

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
x_train,x_test,y_train,y_test=create_feature_sets_labels('pos.txt','neg.txt')
n_nodes_hl1 = 1500
n_nodes_hl2 = 1500
n_nodes_hl3 = 1500

n_classes = 2
batch_size = 100

x = tf.placeholder('float')
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(x_train[0]), n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.initialize_all_variables())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while(i<len(x_train)):
                start=i
                end=i+batch_size
                batch_x=np.array(x_train[start:end])
                batch_y=np.array(y_train[start:end])
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                epoch_loss += c
                i+=batch_size

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))

train_neural_network(x)

423
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 10 loss: 1252320.3803710938
Epoch 2 completed out of 10 loss: 509426.2421875
Epoch 3 completed out of 10 loss: 382757.53594970703
Epoch 4 completed out of 10 loss: 319008.1851196289
Epoch 5 completed out of 10 loss: 270133.49014282227
Epoch 6 completed out of 10 loss: 201454.9569091797
Epoch 7 completed out of 10 loss: 143120.79821395874
Epoch 8 completed out of 10 loss: 123567.68896913528
Epoch 9 completed out of 10 loss: 77697.44762641191
Epoch 10 completed out of 10 loss: 45149.35910320282


InvalidArgumentError: Expected dimension in the range [-1, 1), but got 1
	 [[node ArgMax_1 (defined at <ipython-input-11-0d894a7d65d7>:70) ]]

Errors may have originated from an input operation.
Input Source operations connected to node ArgMax_1:
 Placeholder_1 (defined at <ipython-input-11-0d894a7d65d7>:13)

Original stack trace for 'ArgMax_1':
  File "c:\users\dell\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\dell\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\users\dell\appdata\local\programs\python\python38\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\dell\appdata\local\programs\python\python38\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "c:\users\dell\appdata\local\programs\python\python